In [ ]:
import sys
sys.path.append("..")

from pathlib import Path
import math

from tqdm import tqdm
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as VT
import torchvision.transforms.functional as VF
from torchvision.utils import make_grid

from src.datasets.ca imiiport *
from src.models.rbm import RBM

In [ ]:
ds = TotalCADataset(
    (32, 32), 
    seed=23,
    #num_repetitions=8,
    #num_iterations=[1, 20],
    #init_prob=[0, 1],
    #rules=["3-23", "124-45"],
)
len(ds)

In [ ]:
def plot_images(iterable, num: int = 8, nrow: int = 8):
    images = []
    for image, rule in iterable:
        if isinstance(image, tuple):
            image = image[0]
        images.append(image)
        if len(images) >= num:
            break
    return VF.to_pil_image(make_grid(images, nrow=nrow))

dl = DataLoader(ds)
plot_images(dl, 256, nrow=16)

In [ ]:
model = RBM(math.prod(ds.shape), 32)
model.load_state_dict(torch.load("../checkpoints/ca32x32/snapshot.pt")["state_dict"])

In [ ]:
# store ca -> features

dl = DataLoader(ds, batch_size=100, num_workers=5)
features = []
for batch in tqdm(dl):
    images, rules = batch
    features.append(model(images))
    #if len(features) > 2:
    #    break
features = torch.cat(features)
torch.save(features, "../checkpoints/ca32x32/ca-features.pt")
features.shape

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(2, verbose=True)
xy = tsne.fit_transform(features.detach())

In [ ]:
np.save("../checkpoints/ca32x32.npy", xy)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(3)
rgb = pca.fit_transform(features.detach())
rgb -= rgb.min()
rgb /= rgb.max()
rgb.shape, rgb.min(), rgb.max()

In [ ]:
colors = [
    "#" + "".join(
        "%02x" % max(0, min(255, int(50 + v*205)))
        for v in row
    )
    for row in rgb
]
colors[:3]

In [ ]:
import plotly
import plotly.express as px
plotly.io.templates.default = "plotly_dark"

#plotly.graph_objs.Scattergl(
p = px.scatter(
    x=xy[:, 0], y=xy[:, 1],
    height=1000,
    color=colors,
    color_discrete_map="identity",
)
p.update_layout({"margin": {"l": 0, "r": 0, "t": 0, "b": 0}}, overwrite=True)
p#.layout.